# Julia Fundamentals

- Syntax and some simple I/O
- Variables and Scope, Local and Global, Static and Dynamic, and Constants
- Primitive, Abstract and Composite Types
- Operators and elementary math functions
- Strings (Unicode and UTF-8)
- String operations and pattern matching

## 1. Syntax and Simple I/O

- Julia syntax and expressions that are simple and clear
- Terminal I/O
- Simple file I/O using csv files


#### Shell Game

In [4]:
# Shell Game example (Coconut shells and 1 pea)
# Shells are variable names
# Under each shell, type with value
# This example shows that variable names are just labels
# They can be made concrete and that will be shown later
# type and data go together as the following will illustrate
a = 3
println("shell_a is ", typeof(a))
b = [1 2; 3 4]
println("shell_b is ", typeof(b))
c = "pea"
println("shell_c is ", typeof(c))

println("Before: $a, $b, $c")

# We can shuffle these around 
a, b, c = b, c, a # Shell game
a, b ,c = b, a , c # Where is the pea

println("After: $a, $b, $c")

shell_a is Int64
shell_b is Array{Int64,2}
shell_c is String
Before: 3, [1 2; 3 4], pea
After: pea, [1 2; 3 4], 3


#### How deep is a well?

In [6]:
# How deep is a well? Use a stopwatch
# dropping a rock and listening for the splash

# depth = 0.5g * td^2     td = time to reach depth (s = 0.5a * t^2)
# depth = vs * tu         tu = time sound take from bottom to top

g = 9.8065    # acceleration due to gravity
vs = 343.0    # m/s velocity of sound

print("drop to splash time: ")    # Seconds from drop to ear
st = readline();    #Arg STDIN assumed if left out
t = parse(Float32, st)    #String to Float32 convertion for t


# solve equations by
# EQ1: .5g*td^2 - vs*tu = 0
# EQ2: td+tu = t => tu=t-td substitute in EQ1 and get
# quadratic .5g*td^2 + vs*td - vs*t =0 in t1

td = (-vs + sqrt(vs^2 + 2.0g*vs*t))/g
tu = t - td
depthu = vs*tu
println("depth $depthu meters\n sound time $tu seconds")

#confirm
depthd = .5g*td^2
println("confirming depth $depthd meters\n fall time $td seconds")

drop to splash time: stdin> 4
depth 70.58719849910099 meters
 sound time 0.20579358163003203 seconds
confirming depth 70.58719849910122 meters
 fall time 3.794206418369968 seconds


#### Shopping list (I/O csv)

In [9]:
file_in = open("files/str1.csv")
file_out = open("files/str2.csv", "w")
cnt = 1


while(!eof(file_in))
    global cnt  # We need to specify cnt as global so we can write to it,
                # else it would be interpreted as a variable inside the while loop
    row = readline(file_in)
    row = "$cnt,"*row*"\n"
    write(file_out, row)
    print(row)
    cnt += 1
end
close(file_in)
close(file_out)
println("Done!")

1,salt
2,pepper
3,milk
4,cheese
5,tomatoes
6,potatoes
Done!


#### Takeaways

- Learned how variable names are just labels
- Learned what Julia syntax looks like
- Learned how to communicate with your terminal/console window
- Learned reading and writing CSV files that can be used with Excel or OpenOffice

## 2. Variables and Scope - Local and Global, Static and Dynamic, and Constants

- Global and local variables
- Static and Dynamic
- Constants

##### Scope global vs local

If we want to write to a variable inside of a loop, we need to specify it as global inside such loop.

In [11]:
i=0             # (global)
while i == 0    # Ok since being read
    global i    # because being written below
    x = π       # (soft local) needs global to be seen outside of while
    i+= 1       # Need the global since being written
    println(i)
    println(x)  # x is local
end


1
π


In [12]:
println(i)
println(x)      # error since x is local in while

1


UndefVarError: UndefVarError: x not defined

#### "Static"

"Static" type state becomes what in C would be called a static type.
Its value persists but is only available inside the let block.
Useful for creating pointers in buffers, deques and queues.

In [13]:
let
    state=0
    global counter
    counter() = state+=1
end
println(counter())
println(counter())

1
2


In [14]:
# Loop indices and externally defined variables local

i = 43 # these will NOT be overwritten by the loop indfex and local l
l = 66

for i=1:3   # this i is "soft" local so won't interfere with global i
    
    j=i+1   # j is also "soft" local to this block and lower
    
    for k=1:2
        local l # "Hard" local so won't interfere with global l=66
        l = k + j + i
        println("l ",l)
    end
end

l 4
l 5
l 6
l 7
l 8
l 9


In [19]:
println(i)  # 1:3 won't override 43
println(l)  # note inner for loop l is hard local
println(j)  # error j soft local not available to higher level

43
66


UndefVarError: UndefVarError: j not defined

#### Constants

Constants are like variables, but they will warn you if you override them.

In [23]:
const m = 43.5

43.5

In [24]:
println(m)

43.5


In [25]:
m = 42   # warning error

ErrorException: invalid redefinition of constant m

In [26]:
# Built in constants
π #(alias:pi)

π = 3.1415926535897...

In [27]:
# CODATA.jlexp(l) package has 350 physics constants

#### Takeaways

- Scope rules for variables
- Static variables for shared pointers and counters
- Constants cannot be overriden

## 3. Primitive, abstract and composite types

- Primitives, concrete types, and CPU storage requirements
- Abstract types have no instantiation, but form part of the conceptual hierarchy
- Composite types - a configuration of primitive types and parameters


### Type tree

<img src="files/type-tree.png">


#### Big and Rational

- MPRF.BigFloat &rarr; uses the GNU C MFPR library arbitrary precision package;
    - precision(BigFloat) &rarr; will give current size in # of bits
    - BigFloat($\pi$, 500) &rarr; using 500 binary digits
- GMP.BigInt &rarr; uses the GNU C GMP library, size of numbers limited by memory
    - BigInt(2)^512 &rarr; 2^512 using 513 binary digits (it expands as necessary)
    
#### Complex and Aliases

- ComplexF16 for Complex{Float16}
- ComplexF32 for Complex{Float32}
- ComplexF64 for Complex{Float64}
- CPU dependent aliases:
    - Float &rarr; alias for Float64 or Float32
    - Int &rarr; alias for Int64 or Int32
    - UInt &rarr; alias for UInt64 or UInt32

### Primitive Types

#### Integer types with maxnumber

- Int is default for Int64 if a 64 bit CPU, else 32
- UInt is default for UInt64 if a 64 bit CPU, else 32

In [33]:
println(Int8(127))
println(UInt8(255))

127
255


In [34]:
println(Int16(32767))
println(UInt16(65535))

32767
65535


In [35]:
println(Int32(2147483647))
println(UInt32(4294607295))

2147483647
4294607295


In [36]:
println(Int64(2)^63-1)
println(UInt64(2)^64-1)

9223372036854775807
18446744073709551615


In [37]:
println(Int128(2)^127-1)
println(UInt128(2)^128-1)

170141183460469231731687303715884105727
340282366920938463463374607431768211455


In [38]:
println(BigInt(2)^2046-1)

8079251517827751825178719172167487990111025667428871008032586356881163784716972723299300352880728365922179490230474504873529889787622730273772038096612070780157719341825249022937549437597413026699014409596016892069198054660654939040459523584619042617645411463009076260721893972885266452151888099780982596380478583347417085605171243696641142373714044008831580514519451414832756548177115078537564648216044279181485900929615464339399587788075411476100924403308321807806781421177705052431289275431732830867419635645164174483761499317088249659553881291597359333885900533858307401161329619651238037048388963402764899057663


#### Floating point types and irrational numbers

In [39]:
println(Float16(π))    # 11 bits of precision
println(Float32(π))    # 24 bits of precision
println(Float64(π))    # 53 bits of precision
println(BigFloat(π))   # 256 bits of precision - normally

3.14
3.1415927
3.141592653589793
3.141592653589793238462643383279502884197169399375105820974944592307816406286198


In [40]:
precision(BigFloat(π))

256

In [42]:
println(π)    # irrational number cannot be represented as a rational

π


In [45]:
BigFloat(π)   # BigFloat defaults to 256 binary places ~77 decimal places

3.141592653589793238462643383279502884197169399375105820974944592307816406286198

In [46]:
BigFloat(π, 500)    # 500+ bits of precision

3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679821480865132823066470938446095505822317253594081283

#### Complex Numbers

In [47]:
println(Complex{Float16}(π + 2π*im))    # 11 bits precision each part
println(Complex{Float32}(π + 2π*im))    # 24 bits precision each part
println(Complex{Float64}(π + 2π*im))    # 53 bits precision each part

Float16(3.14) + Float16(6.28)im
3.1415927f0 + 6.2831855f0im
3.141592653589793 + 6.283185307179586im


##### full gamut of elementary functions work on complex numbers
#### complex shortcuts

In [49]:
ComplexF16(3.1 + 3.2im)    # Alias for Complex{Float16}()

Float16(3.1) + Float16(3.2)im

In [50]:
ComplexF32(3.1 + 3.2im)    # Alias for Complex{Float32}()

3.1f0 + 3.2f0im

In [51]:
ComplexF64(3.1 + 3.2im)    # Alias for Complex{Float64}()

3.1 + 3.2im

In [52]:
Complex(3.1 + 3.2im)       # Alias for ComplexF64 if 64 bit CPU, else ComplexF32

3.1 + 3.2im

### Rational numbers

Used for exact arithmetic

In [54]:
# for example: 1/3 cannot be represented exactly by any
# floating point number.
# 1//3 represents the rational number 1/3 exactly

a = 1//3 + 2//5 - 3//7
b = (22//7) * (5//13)
c = ((22//7)/2)^2

print("a=$a, b=$b, c=$c")

a=32//105, b=110//91, c=121//49

In [55]:
Rational(3.875)    # Make a rational fraction out of a float

31//8

In [56]:
Rational(3.825)    # a repeating binary pattern so not exact, use rationalize() function

4306567143673037//1125899906842624

In [58]:
BigFloat(3.825)    # Gives wrong precision due to prior conversion error

3.82500000000000017763568394002504646778106689453125

In [60]:
BigFloat(3825//1000) # or

3.825000000000000000000000000000000000000000000000000000000000000000000000000007

In [61]:
BigFloat("3.825")

3.825000000000000000000000000000000000000000000000000000000000000000000000000007

##### Takeaways

- Learned abstract, concrete and derived types
- Learned about the default types Flt, Int, UInt, and machine dependence/independence
- Learned Complex, Rationals, BigInts and BigFloats

## 4. Operators and Elementary Math Functions

- List of operators:
    - Operations are determined by the context in which they are used (Polymorphism)
- Elementary Math Functions work for
    - Integers (Signed and Unsigned; Int8-Int128 and Uint8-Uint128, BigInt)
    - Floating Point (Float16, Float32, Float64, BigFloat)
    - Complex (ComplexF16, ComplexF32, ComplexF64)
    - Rationals
- Vectors and Matrices


<img src='files/math-operators.png'>

**Y = Yes**

##### Function Argument Types

- Types Integer, Float, Rational can all be automatically converted to Float
- This also includes Complex types that make sense and produce complex results
- It's a good idea to put rational and complex numbers in parenthesis
    - y = x^3//2 and z = x*1+3im are not the same as y = x^(3//2) and z = x*(1+3im)
    - The former can confuse the compiler and give unintended result

#### Operators  +  -  *  /  |  ^

In [65]:
#e = Base.MathConstants.e

a = 2//3 + 3.4π/e

4.5936574836539075

In [66]:
b = (3//4)^2 - 55\5   # x=a/b  is x=b\a

0.47159090909090906

In [67]:
V = [1, 2, 3]         # vector

3V

3-element Array{Int64,1}:
 3
 6
 9

In [68]:
M = [1 2; 3 4]        # 2x2 matrix

Y = 3M                # * implied, 3* each entry

2×2 Array{Int64,2}:
 3   6
 9  12

In [69]:
Z1 = M/3              # Matrix M divided by 3

2×2 Array{Float64,2}:
 0.333333  0.666667
 1.0       1.33333

In [70]:
Z2 = 3\M              # Divide into -- same result as above

2×2 Array{Float64,2}:
 0.333333  0.666667
 1.0       1.33333

In [71]:
W1 = (3+3im)\M

2×2 Array{Complex{Float64},2}:
 0.166667-0.166667im  0.333333-0.333333im
      0.5-0.5im       0.666667-0.666667im

In [72]:
W2 = (3+3im)W1

2×2 Array{Complex{Float64},2}:
 1.0+0.0im  2.0+0.0im
 3.0+0.0im  4.0+0.0im

##### op= Operators

In [73]:
i=0
i+=1

1

In [74]:
M*=2   # Stays integer, all entries times 2

2×2 Array{Int64,2}:
 2  4
 6  8

In [75]:
M.-=1  # Stays integer, -1 from each entry

2×2 Array{Int64,2}:
 1  3
 5  7

In [76]:
M*=2.0 # now floating, all entries * 2.0

2×2 Array{Float64,2}:
  2.0   6.0
 10.0  14.0

##### Special functions for complex numbers

In [77]:
real(1+2im)  # real part

1

In [78]:
imag(1+2im)  # imaginary part

2

In [80]:
conj(1+2im)  # conjugate

1 - 2im

##### Basic function library

In [81]:
d=3+3im

3 + 3im

In [82]:
a = sqrt(d)  # square-root

1.9029767059950162 + 0.7882387605032136im

In [83]:
a^2

3.0 + 3.0im

In [84]:
# notice sqrt(-1) gives an error
a=sqrt(-1)

DomainError: DomainError with -1.0:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [85]:
# but sqrt(-1 + 0im) works
a=sqrt(-1 + 0im)

0.0 + 1.0im

In [88]:
a1=abs(1-2im)

2.23606797749979

In [87]:
a2=abs2(1+2im)  # abs2 = multiply complex number by its conjugate

5

In [89]:
a3 = sqrt(-1+0im)

0.0 + 1.0im

In [90]:
a4 = sqrt(-1+2im)

0.7861513777574233 + 1.272019649514069im

In [91]:
a5 = sin(1+2im)

3.165778513216168 + 1.9596010414216063im

In [92]:
a6 = cosh(1+2im)  # hiperbolic cosine

-0.64214812471552 + 1.0686074213827783im

In [93]:
a7 = exp(1+2im)

-1.1312043837568135 + 2.4717266720048188im

In [94]:
a8 = log(1+2im)

0.8047189562170501 + 1.1071487177940904im

#### Miscellaneous functions for Ints and Floats

In [97]:
# also round(x), floor(x), ceil(x), trunc(x)
# rem(x,y), mod(x,y), gcd(x,y) greatest common denominator,
# lcm(x,y) least common multiple and many Variations

# Also hypot(x,y) hypotenuse, crbt(x) cube root,
# various log and exp functions

println(ceil(4.3))
println(round(4.3))
println(floor(4.3))
println(hypot(3,4))
println(cbrt(27))

5.0
4.0
4.0
5.0
3.0


#### Takeaways

- Learned how operators are polymorphic across a broad spectrum of types
- A large number of functions share this polymorphism as well if it makes sense for the funcion's arguments

## 5. Strings (Unicode and UTF8)

